# Biome Level statistics for model: SoilCarbon


In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [2]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()

In [3]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites and images

In [4]:
# load the basic maps that needed for the analysis
# load the two composites tha will be used in the analysis
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
biomeMask = biomeLayer.mask(biomeLayer.neq(98)).mask(biomeLayer.neq(99)).gt(0)
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000)
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)
# define the standard projection
stdProj = biomeLayer.projection()

## 2 Load the biomass density maps

In [5]:
# load the carbon density layers
SandermannCarbonDiff = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU").unmask()
SandermannCarbonPresent = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_1km_Present").unmask()

# mask the diffrence layer
SandermannCarbonLoss = SandermannCarbonDiff.multiply(SandermannCarbonDiff.gt(0))

# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask = presentForestCover.gt(0)
potentialMask = potentialCoverAdjusted.gte(0.1)

# calculate the sum of the potential in soil with the consideration of forest cover
SandermannCarbonStockLoss = SandermannCarbonLoss.multiply(pixelAreaMap).divide(1000000000).mask(biomeMask).mask(potentialMask).multiply(potentialCoverAdjusted)

SandermannCarbonStockLossResult =SandermannCarbonStockLoss.reduceRegion(reducer = ee.Reducer.sum(),
                                                                        geometry = unboundedGeo,
                                                                        crs = 'EPSG:4326',
                                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                                        maxPixels = 1e9)

print('Soil potential sum',SandermannCarbonStockLossResult.getInfo())

Soil potential sum {'b1': 49.30515651720103}


## 4 Partioning the potential cover into different landuse types

In [6]:
# Load all the landuse type layers
croplandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/cropland_Percent").rename('cropland').divide(100).reproject(crs=stdProj);
grazingOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/grazing_Percent").rename('grazing').divide(100).reproject(crs=stdProj);
pastureOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/pasture_Percent").rename('pasture').divide(100).reproject(crs=stdProj);
rangelandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/rangeland_Percent").rename('rangeland').divide(100).reproject(crs=stdProj);
urbanOrg = compositeImage.select(['LandCoverClass_Urban_Builtup']).divide(100).unmask().reproject(crs=stdProj);
snowIceOrg = compositeImageNew.select(['ConsensusLandCoverClass_Snow_Ice']).divide(100).unmask().reproject(crs=stdProj);
openWaterOrg = compositeImageNew.select(['ConsensusLandCoverClass_Open_Water']).divide(100).unmask().reproject(crs=stdProj);
# define the total landcover types
sumCover = presentForestCover.add(pastureOrg).add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(openWaterOrg).add(snowIceOrg);
oneSubtract = ee.Image(1).subtract(sumCover);
freeland = oneSubtract.multiply(oneSubtract.gte(0));
# get the scale ratio for pixels with sumCover larger than 1
scaleRatio = ee.Image(1).subtract(presentForestCover).divide(sumCover.subtract(presentForestCover)).multiply(oneSubtract.lt(0));
# get the ratio of these three disturbed maps
pasture = pastureOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(pastureOrg.multiply(oneSubtract.gte(0))).unmask();
rangeland = rangelandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(rangelandOrg.multiply(oneSubtract.gte(0))).unmask();
cropland = croplandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(croplandOrg.multiply(oneSubtract.gte(0))).unmask();
urban = urbanOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(urbanOrg.multiply(oneSubtract.gte(0))).unmask();
openWater = openWaterOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(openWaterOrg.multiply(oneSubtract.gte(0))).unmask();
snowIce = snowIceOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(snowIceOrg.multiply(oneSubtract.gte(0))).unmask();
sumTT = presentForestCover.add(pasture).add(rangeland).add(cropland).add(urban).add(freeland).add(openWater).add(snowIce).unmask();

effectivePotentialMask = freeland.add(rangeland).add(pasture).add(cropland).add(urban).gt(0);
# there are some pixels without any landcover survived but with open water and ice and snow. here we mask these pixels out
sumlandCover = pastureOrg.add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(freeland);
restorationMap = potentialCoverAdjusted.subtract(presentForestCover).mask(effectivePotentialMask).unmask();

# sum all these scaled layersv
scaledSum = pasture.add(rangeland).add(cropland).add(urban).add(freeland);
potentialCoverFinal = restorationMap.add(presentForestCover);
# allocate the potential equally to each layer
freelandPotentialCover = freeland.divide(scaledSum).multiply(restorationMap).unmask();
rangelandPotentialCover = rangeland.divide(scaledSum).multiply(restorationMap).unmask();
pasturePotentialCover = pasture.divide(scaledSum).multiply(restorationMap).unmask();
croplandPotentialCover = cropland.divide(scaledSum).multiply(restorationMap).unmask();
urbanPotentialCover = urban.divide(scaledSum).multiply(restorationMap).unmask();
#  allocate the freeland potential in pixels with forest cover larger than 10% to conservation potential
freelandForConsevation = freelandPotentialCover.multiply(presentForestCover.gte(0.1)).unmask();
maximumPotentialCover = freelandForConsevation.add(presentForestCover);
# calucate the reall freeland outside of forest
freelandLeftMap = freelandPotentialCover.subtract(freelandForConsevation).unmask()# the left positive pixels are real freeland pixels

## 5 Partioning the biomass potential into different landuse types

In [7]:
# calculate the existing carbon, present potential carbon and absolute potential carbon in forests
# absoluteImage1 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(presentForestCover).divide(1000000000).mask(potentialMask)
absoluteImage2 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(maximumPotentialCover).divide(1000000000).mask(potentialMask).rename('ConservationPotential')
absoluteImage3 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(potentialCoverFinal).divide(1000000000).mask(potentialMask).rename('AbsolutePotential')
absoluteImage4 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(freelandLeftMap).divide(1000000000).mask(potentialMask).rename('FreelandPotential')
absoluteImage5 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(rangelandPotentialCover).divide(1000000000).mask(potentialMask).rename('RangelandPotential')
absoluteImage6 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(pasturePotentialCover).divide(1000000000).mask(potentialMask).rename('PasturePotential')
absoluteImage7 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(croplandPotentialCover).divide(1000000000).mask(potentialMask).rename('CroplandPotential')
absoluteImage8 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(urbanPotentialCover).divide(1000000000).mask(potentialMask).rename('UrbanPotential')
freelandForConservation = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(freelandForConsevation).divide(1000000000).mask(potentialMask).rename('FreeToConservation')
# absoluteImage9 = SandermannCarbonLoss.multiply(pixelAreaMap).multiply(freelandForConsevation).divide(1000000000).mask(potentialMask)

## Calculate the potential numbers and write into local folder

In [10]:
# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage2.addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['ConservationPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/SoilCarbon_Biome_Level_Statistics.csv',header=True,mode='w+')
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,ConservationPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,8.975195,12.904450,0.675013,0.007705,1.483885,1.710005,0.052648,1.025816
1,0.821169,1.936013,0.274057,0.002188,0.279677,0.552029,0.006892,0.125954
2,0.420925,0.618023,0.058311,0.002971,0.080813,0.054471,0.000532,0.070388
3,8.214611,15.119828,1.322182,0.004839,1.985723,3.434442,0.158030,0.977793
4,1.284450,1.771870,0.151794,0.005222,0.183607,0.135407,0.011390,0.232313
5,1.190913,1.448248,0.116017,0.000116,0.054780,0.083797,0.002625,0.156872
6,4.116150,7.517334,0.473929,1.327168,0.924321,0.666500,0.009266,0.535046
7,0.673453,3.732070,0.454001,0.893977,0.464055,1.226332,0.020253,0.059060
8,0.241434,0.452222,0.035493,0.105897,0.043868,0.023774,0.001756,0.020211
9,0.399438,0.816783,0.069650,0.122722,0.109976,0.114243,0.000754,0.044111


In [95]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.